In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

In [2]:
# import spacy
# nlp = spacy.load('en_core_web_lg')
# def tokenizer(corpus):
#     return [str(token) for token in nlp(corpus)]
# a = '"River walk that led me to my secret family: after being adopted as a child, Katharine Norbury reveals the emotional journey to reconnect with her biological mother"'.replace('"', '')
# tokenizer(a)

## Load and prepare data

In [3]:
pretrained_file = 'glove.840B.300d'
train_file = 'data/train.csv'
test_file = 'data/test.csv'
max_size = 64
min_freq = 5
batch_size = 128

In [4]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, max_size, min_freq, batch_size)

In [5]:
# AttractiveData.LABEL.vocab.freqs

In [6]:
for i, sentence in enumerate(AttractiveData.test_data):
    if i == 3:
        print(vars(AttractiveData.train_data[i]), vars(sentence))

{'Headline': ['sorry', ',', 'i', 'spent', 'it', 'on', 'myself', '!', 'harvey', 'nichols', "'", 'hilarious', 'christmas', 'advert', 'sees', 'people', 'treating', 'themselves', 'instead', 'of', 'others'], 'Category': 'femail', 'Label': '3.3333333333333335'} {'Headline': ['three', 'police', 'officers', 'accused', 'of', 'stealing', '?', '?', '30k', 'during', 'raid', 'on', 'criminal'], 'Category': 'news', 'Label': ''}


In [7]:
a = next(iter(AttractiveData.trainloader))
for i in range(len(a)):
    print(a.Headline[i], a.Label[i])
    1/0

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor(4.6667, device='cuda:0')


ZeroDivisionError: division by zero

## Start to train

In [8]:
timestr = time.strftime("%Y%m%d-%H%M%S")
save_name = './model/AttractiveNet'
num_workers = 10
input_dim = len(AttractiveData.TEXT.vocab)
embedding_dim = 300
hidden_dim = 128
output_dim = 1
log_steps = 10
epochs = 100
lr = 1e-5
num_layers = 2
nhead = 4
dropout = 0.1
pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([1432, 300])


In [9]:
# max(AttractiveData.df_train.Headline.str.len()), max(AttractiveData.df_test.Headline.str.len())

In [10]:
AttractiveTrainer = AttractiveTrainer(save_name, log_steps, epochs, lr, timestr, AttractiveData.device, AttractiveData.trainloader, AttractiveData.testloader, input_dim, embedding_dim, hidden_dim, output_dim, pretrained_embeddings, dropout, num_layers, nhead)

In [11]:
AttractiveTrainer.model

TransformerModel(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1432, 300, padding_idx=0)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_fe

In [12]:
AttractiveTrainer.train()


EP:6 | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 76.94it/s]
EP: train | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 177.39it/s]
EP:7 | lr: 1e-05:  50%|| 8/16 [00:00<00:00, 75.00it/s]
EP_train | avg_loss: 2.8246096819639206 |
EP:7 | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 75.40it/s]
EP: train | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 158.32it/s]
EP:8 | lr: 1e-05:  50%|| 8/16 [00:00<00:00, 73.95it/s]
EP_train | avg_loss: 2.5004172921180725 |
EP:8 | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 74.40it/s]
EP: train | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 170.88it/s]
EP:9 | lr: 1e-05:  50%|| 8/16 [00:00<00:00, 77.13it/s]
EP_train | avg_loss: 2.2228206992149353 |
EP:9 | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 76.67it/s]
EP: train | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 168.92it/s]
EP:10 | lr: 1e-05:  50%|| 8/16 [00:00<00:00, 76.14it/s]
EP_train | avg_loss: 1.9791988208889961 |
EP:10 | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 76.07it/s]
EP: train | lr: 1e-05: 100%|| 16/16 [00:00<00:00, 174.91it/s]
EP:11 | lr: 1e-05: 

In [11]:
AttractiveTrainer.train()

EP:0 | lr: 0.001:   0%|| 0/32 [00:00<?, ?it/s]tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [  0,   0, 288,  ..., 526,   0,   0],
        [849,  15, 458,  ...,   0,   8, 300],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0')
tensor([ 0.,  2., 13.,  7.,  4.,  3.,  2.,  1.,  1.,  2.,  5., 11.,  0.,  1.,
         6.,  0.,  1.,  6.,  4.,  3.,  1.,  0.,  3.,  3.,  2.,  6.,  0.,  2.,
         9.,  4., 12.,  7.,  6.,  1.,  1.,  2.,  3.,  1.,  3.,  5., 11.,  5.,
         2.,  9., 11.,  0.,  1.,  6.,  0.,  0.,  8.,  1.,  5.,  0.,  9.,  4.,
        13.,  3.,  2.,  2.,  1.,  3.,  0.,  6.], device='cuda:0')
tensor([[[ 6.0121e-01],
         [ 3.5993e-01],
         [ 2.6935e-01],
         ...,
         [ 9.3061e-01],
         [ 6.7218e-01],
         [ 5.9746e-01]],

        [[ 5.3440e-01],
         [ 5.7120e-01],
         [ 9.3283e-01],
         ...,
         [ 2.4973e-01],

ZeroDivisionError: division by zero

In [11]:
AttractiveTrainer.train()

EP:0 | lr: 0.001:   0%|| 0/32 [00:00<?, ?it/s]tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [ 48, 241,   0,  ...,   0, 805,   0],
        [211,   3,   0,  ...,   0,  13, 303],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0')
tensor([ 4.,  1.,  2.,  2.,  0., 13.,  3.,  9.,  1.,  1.,  3.,  3.,  2.,  1.,
         2.,  2.,  0.,  8.,  1.,  9.,  0.,  1.,  0.,  3.,  6., 10.,  2.,  7.,
         4.,  1.,  2.,  2.,  6.,  6.,  2., 14.,  2.,  3.,  7.,  0.,  4.,  3.,
         1., 13.,  0.,  0.,  1.,  1.,  1.,  0.,  4.,  2.,  5.,  1.,  3.,  5.,
         0.,  2.,  0.,  8.,  1.,  0.,  9.,  1.], device='cuda:0')
tensor([[-0.2539],
        [-0.8596],
        [-1.1908],
        [-1.5457],
        [-0.2728],
        [-0.9879],
        [-0.8742],
        [-1.1552],
        [-1.1962],
        [-1.4770],
        [-0.5963],
        [-1.6508],
        [-0.5904],
        [-1.2092],
   

ZeroDivisionError: division by zero

In [13]:
def predict_attractive(sentence):
    tokens = AttractiveData.tokenizer(sentence)
    indexed = [AttractiveData.TEXT.vocab.stoi[t] for t in tokens]
    # print(indexed)
    tensor = torch.LongTensor(indexed).to(AttractiveData.device)

    tensor = tensor.unsqueeze(1)
    # print(tensor.shape)
    prediction = AttractiveTrainer.model(tensor)

    # print(prediction.shape)
    
    return prediction

In [14]:
a = "Cancer left Eric With half his face missing and unable to eat or drink. now surgeons have made him a new face using a 3d printer"
prediction = predict_attractive(a)
prediction.item()

2.964232921600342

## Below is testing

In [16]:
next(iter(AttractiveData.testloader))

AttributeError: 'Example' object has no attribute 'Text'

In [15]:
test_headline = AttractiveData.df_test['Headline'].to_list()
predict_list = []
for i in range(len(test_headline)):
    predict_list.append(predict_attractive(test_headline[i]).item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv('transformers.csv', index=False)

## Below just for fun guess

In [17]:
a = AttractiveData.df_test['ID'].to_list()

In [17]:
from sklearn.metrics import mean_squared_error

In [9]:
mean_squared_error(a, b)
# Training all 3.0 got mse = 0.5545

0.5545343137254902

In [18]:
# only for fun
import random
guess_list = [2.3333333333333335, 3.3333333333333335, 3.6666666666666665, 2.6666666666666665]
b = []
for i in range(len(a)):
    b.append(random.choice(guess_list))

In [19]:
AttractiveData.df_test['Label'] = b

In [20]:
AttractiveData.df_test[['ID', 'Label']].to_csv('all_3.csv', index=False)